In [25]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os
import regex

In [22]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 300)

In [34]:
DATA_PATH = '../data/'
listings = DATA_PATH + os.listdir('../data/')[0]
new_listings = DATA_PATH + os.listdir('../data/')[1]
home_values = DATA_PATH + os.listdir('../data/')[2]


In [35]:
listings_df = pd.read_csv(listings)
listings_df

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31
0,102001,0,United States,country,NaN,1421528.0,1500193.0,1592413.0,1660613.0,1709142.0,1733387.0,1723097.0,1701371.0,1642145.0,1541994.0,1452603.0,1405515.0,1449965.0,1515570.0,1604227.0,1665082.0,1693555.0,1688690.0,1665421.0,1634586.0,1560531.0,1441428.0,1336187.0,1285461.0,1309043.0,1299807.0,1317974.0,1320181.0,1345172.0,1324973.0,1283020.0,1228208.0,1140490.0,1028143.0,949576.0,881616.0,882081.0,882293.0,934109.0,978002.0,1028524.0,1071734.0,1083989.0,1067840.0,997847.0,887059.0,777085.0,706905.0,726913.0,774472.0,860326.0,946905.0,1033311.0,1092069.0,1101482.0,1085079.0,1032570.0,955889.0,884129.0,830977.0,835429.0,845834.0,880510.0,907228.0,930911.0,950311.0,959735.0,977241.0,970238.0,932240.0,890491.0,876353.0,913266.0
1,394913,1,"New York, NY",msa,NY,73707.0,80345.0,85864.0,90067.0,91881.0,91252.0,90050.0,89340.0,87186.0,81944.0,76678.0,74497.0,78192.0,84211.0,91255.0,95966.0,97260.0,95383.0,93359.0,91309.0,87828.0,80727.0,74301.0,72005.0,74371.0,72270.0,70474.0,70352.0,77012.0,81547.0,82809.0,80976.0,77001.0,70532.0,65541.0,61868.0,61966.0,62738.0,65965.0,68986.0,70650.0,70786.0,69101.0,67467.0,63984.0,57820.0,50768.0,46430.0,48039.0,51340.0,55710.0,58872.0,61312.0,61386.0,60262.0,58241.0,55450.0,50400.0,45916.0,42796.0,43253.0,44197.0,45829.0,46198.0,45506.0,43956.0,42515.0,42586.0,42068.0,39540.0,36461.0,34858.0,36219.0
2,753899,2,"Los Angeles, CA",msa,CA,21998.0,23784.0,25605.0,27109.0,28811.0,29874.0,30428.0,30606.0,30035.0,28252.0,26635.0,25849.0,27132.0,28454.0,30124.0,30980.0,31524.0,31208.0,30618.0,29591.0,27588.0,24357.0,21649.0,20624.0,21194.0,20897.0,21255.0,22209.0,24174.0,25175.0,25453.0,25306.0,24128.0,21871.0,19808.0,18781.0,19497.0,20038.0,20588.0,20766.0,21361.0,21816.0,21609.0,20711.0,18955.0,16307.0,14161.0,13376.0,14653.0,16023.0,17635.0,19187.0,21015.0,22264.0,22400.0,21802.0,20477.0,18604.0,16767.0,15382.0,15214.0,15103.0,15283.0,15378.0,15689.0,15969.0,16032.0,16167.0,15877.0,14897.0,14058.0,13754.0,14558.0
3,394463,3,"Chicago, IL",msa,IL,38581.0,42253.0,45757.0,47492.0,48985.0,49782.0,49630.0,48916.0,46495.0,42304.0,38250.0,36754.0,39387.0,43789.0,48113.0,50794.0,51883.0,52135.0,51829.0,50805.0,47938.0,43093.0,38966.0,37740.0,39339.0,39363.0,39734.0,40277.0,42454.0,43099.0,42308.0,40961.0,38027.0,33837.0,31354.0,29581.0,30715.0,31386.0,34006.0,36266.0,38328.0,39729.0,39806.0,38727.0,35528.0,30559.0,25580.0,22343.0,22775.0,24246.0,26775.0,28894.0,30846.0,31876.0,31682.0,30798.0,28771.0,25674.0,22770.0,20829.0,20922.0,21214.0,22091.0,22706.0,23228.0,23568.0,23569.0,23707.0,22957.0,21135.0,19092.0,18342.0,19049.0
4,394514,4,"Dallas, TX",msa,TX,24043.0,25876.0,28225.0,30490.0,32408.0,33567.0,33512.0,32589.0,31265.0,29283.0,27727.0,27057.0,28543.0,30513.0,32932.0,34785.0,36078.0,36285.0,35472.0,34244.0,32494.0,30122.0,28058.0,27264.0,27993.0,28231.0,29129.0,29893.0,31000.0,30306.0,28526.0,26449.0,24033.0,21385.0,19294.0,17682.0,17931.0,18591.0,20030.0,21087.0,22496.0,23661.0,23945.0,23080.0,21361.0,19025.0,16641.0,14913.0,14911.0,15053.0,16146.0,17845.0,20485.0,22365.0,22863.0,22445.0,21500.0,19985.0,18560.0,17106.0,17053.0,17959.0,19987.0,21744.0,22748.0,23251

## Objective
---
- Get listings in North Carolina
- Maybe bring it down to listings near Charlotte
- Get the price 

In [36]:
listings_df_nc = listings_df[listings_df['StateName'] == 'NC']
listings_df_nc.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31
23,394458,23,"Charlotte, NC",msa,NC,9209.0,9880.0,10726.0,11295.0,11824.0,12085.0,11814.0,11399.0,10607.0,9696.0,9035.0,8786.0,9406.0,9921.0,10665.0,10926.0,11137.0,10907.0,10634.0,10294.0,9634.0,8653.0,7879.0,7554.0,8010.0,8117.0,8452.0,8219.0,8060.0,7635.0,7347.0,7117.0,6560.0,5764.0,5419.0,5495.0,6142.0,6494.0,6965.0,7386.0,7910.0,8368.0,8516.0,8495.0,8112.0,7421.0,6648.0,6013.0,6072.0,6313.0,6776.0,7290.0,7970.0,8654.0,8864.0,8689.0,8188.0,7558.0,7198.0,7152.0,7597.0,7811.0,7918.0,7866.0,7799.0,7799.0,7790.0,7878.0,7810.0,7484.0,7175.0,7143.0,7646.0
42,395012,43,"Raleigh, NC",msa,NC,3776.0,4199.0,4608.0,4824.0,4876.0,4803.0,4593.0,4517.0,4327.0,4082.0,3894.0,3865.0,4167.0,4423.0,4804.0,5038.0,5048.0,4926.0,4768.0,4752.0,4532.0,4099.0,3767.0,3638.0,3897.0,3959.0,4083.0,3946.0,3879.0,3674.0,3527.0,3428.0,3164.0,2773.0,2405.0,2151.0,2228.0,2368.0,2575.0,2610.0,2641.0,2709.0,2817.0,2852.0,2688.0,2368.0,1972.0,1767.0,1850.0,2123.0,2540.0,3002.0,3434.0,3800.0,3944.0,3912.0,3681.0,3299.0,2993.0,2756.0,2756.0,2706.0,2720.0,2729.0,2750.0,2816.0,2856.0,2961.0,2925.0,2828.0,2676.0,2671.0,2774.0
75,394648,76,"Greensboro, NC",msa,NC,2946.0,3122.0,3381.0,3472.0,3511.0,3252.0,3015.0,2785.0,2681.0,2553.0,2450.0,2370.0,2436.0,2451.0,2535.0,2573.0,2628.0,2632.0,2612.0,2549.0,2443.0,2258.0,2115.0,2007.0,2031.0,2026.0,2106.0,2112.0,2108.0,2023.0,1987.0,1974.0,1903.0,1776.0,1644.0,1520.0,1523.0,1604.0,1797.0,1937.0,2044.0,2155.0,2220.0,2256.0,2145.0,1958.0,1728.0,1509.0,1449.0,1454.0,1613.0,1772.0,1948.0,2054.0,2075.0,2043.0,1975.0,1836.0,1758.0,1644.0,1654.0,1609.0,1652.0,1658.0,1674.0,1682.0,1703.0,1767.0,1801.0,1778.0,1713.0,1633.0,1627.0
88,395235,89,"Winston, NC",msa,NC,2795.0,3054.0,3354.0,3435.0,3500.0,3278.0,3096.0,2862.0,2740.0,2540.0,2355.0,2254.0,2372.0,2487.0,2596.0,2592.0,2577.0,2551.0,2552.0,2535.0,2448.0,2260.0,2118.0,2017.0,2093.0,2071.0,2065.0,1954.0,1897.0,1847.0,1840.0,1850.0,1791.0,1651.0,1513.0,1395.0,1436.0,1574.0,1776.0,1901.0,1952.0,2043.0,2106.0,2139.0,2038.0,1850.0,1615.0,1439.0,1435.0,1513.0,1656.0,1759.0,1858.0,1952.0,1987.0,1968.0,1866.0,1720.0,1658.0,1622.0,1711.0,1699.0,1693.0,1630.0,1601.0,1614.0,1646.0,1702.0,1695.0,1613.0,1508.0,1419.0,1409.0
94,394549,95,"Durham, NC",msa,NC,1678.0,1858.0,2030.0,2065.0,2090.0,2092.0,2040.0,1998.0,1888.0,1754.0,1615.0,1550.0,1676.0,1868.0,2120.0,2236.0,2217.0,2161.0,2114.0,2146.0,2053.0,1889.0,1697.0,1619.0,1674.0,1690.0,1754.0,1736.0,1744.0,1672.0,1597.0,1542.0,1431.0,1270.0,1108.0,1001.0,1031.0,1072.0,1142.0,1168.0,1197.0,1237.0,1258.0,1279.0,1201.0,1083.0,936.0,861.0,918.0,1024.0,1194.0,1341.0,1498.0,1612.0,1646.0,1629.0,1542.0,1414.0,1269.0,1163.0,1194.0,1242.0,1314.0,1325.0,1331.0,1348.0,1362.0,1404.0,1376.0,1307.0,1203.0,1191.0,1233.0


In [37]:
listings_df_nc['City'] = listings_df_nc['RegionName'].str.extract(r'^(.*),\s*')
listings_df_nc['City']

C:\Users\riain\AppData\Local\Temp\ipykernel_31148\3778743253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df_nc['City'] = listings_df_nc['RegionName'].str.extract(r'^(.*),\s*')


23            Charlotte
42              Raleigh
75           Greensboro
88              Winston
94               Durham
108        Fayetteville
120           Asheville
149             Hickory
172          Wilmington
224        Jacksonville
255          Greenville
256          Burlington
299         Rocky Mount
336            New Bern
343           Lumberton
345           Goldsboro
389              Shelby
392           Pinehurst
448              Wilson
477          Mount Airy
483       Morehead City
486      Roanoke Rapids
493    North Wilkesboro
507         Forest City
514             Sanford
518           Albemarle
546           Cullowhee
560             Kinston
569               Boone
574      Elizabeth City
648          Washington
653              Marion
661          Rockingham
668           Henderson
736    Kill Devil Hills
767          Laurinburg
783             Brevard
Name: City, dtype: object

In [38]:
new_listings_data = pd.read_csv(new_listings)
new_listings_data

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-02-03,2018-02-10,2018-02-17,2018-02-24,2018-03-03,2018-03-10,2018-03-17,2018-03-24,2018-03-31,2018-04-07,2018-04-14,2018-04-21,2018-04-28,2018-05-05,2018-05-12,2018-05-19,2018-05-26,2018-06-02,2018-06-09,2018-06-16,2018-06-23,2018-06-30,2018-07-07,2018-07-14,2018-07-21,2018-07-28,2018-08-04,2018-08-11,2018-08-18,2018-08-25,2018-09-01,2018-09-08,2018-09-15,2018-09-22,2018-09-29,2018-10-06,2018-10-13,2018-10-20,2018-10-27,2018-11-03,2018-11-10,2018-11-17,2018-11-24,2018-12-01,2018-12-08,2018-12-15,2018-12-22,2018-12-29,2019-01-05,2019-01-12,2019-01-19,2019-01-26,2019-02-02,2019-02-09,2019-02-16,2019-02-23,2019-03-02,2019-03-09,2019-03-16,2019-03-23,2019-03-30,2019-04-06,2019-04-13,2019-04-20,2019-04-27,2019-05-04,2019-05-11,2019-05-18,2019-05-25,2019-06-01,2019-06-08,2019-06-15,2019-06-22,2019-06-29,2019-07-06,2019-07-13,2019-07-20,2019-07-27,2019-08-03,2019-08-10,2019-08-17,2019-08-24,2019-08-31,2019-09-07,2019-09-14,2019-09-21,2019-09-28,2019-10-05,2019-10-12,2019-10-19,2019-10-26,2019-11-02,2019-11-09,2019-11-16,2019-11-23,2019-11-30,2019-12-07,2019-12-14,2019-12-21,2019-12-28,2020-01-04,2020-01-11,2020-01-18,2020-01-25,2020-02-01,2020-02-08,2020-02-15,2020-02-22,2020-02-29,2020-03-07,2020-03-14,2020-03-21,2020-03-28,2020-04-04,2020-04-11,2020-04-18,2020-04-25,2020-05-02,2020-05-09,2020-05-16,2020-05-23,2020-05-30,2020-06-06,2020-06-13,2020-06-20,2020-06-27,2020-07-04,2020-07-11,2020-07-18,2020-07-25,2020-08-01,2020-08-08,2020-08-15,2020-08-22,2020-08-29,2020-09-05,2020-09-12,2020-09-19,2020-09-26,2020-10-03,2020-10-10,2020-10-17,2020-10-24,2020-10-31,2020-11-07,...,2021-05-29,2021-06-05,2021-06-12,2021-06-19,2021-06-26,2021-07-03,2021-07-10,2021-07-17,2021-07-24,2021-07-31,2021-08-07,2021-08-14,2021-08-21,2021-08-28,2021-09-04,2021-09-11,2021-09-18,2021-09-25,2021-10-02,2021-10-09,2021-10-16,2021-10-23,2021-10-30,2021-11-06,2021-11-13,2021-11-20,2021-11-27,2021-12-04,2021-12-11,2021-12-18,2021-12-25,2022-01-01,2022-01-08,2022-01-15,2022-01-22,2022-01-29,2022-02-05,2022-02-12,2022-02-19,2022-02-26,2022-03-05,2022-03-12,2022-03-19,2022-03-26,2022-04-02,2022-04-09,2022-04-16,2022-04-23,2022-04-30,2022-05-07,2022-05-14,2022-05-21,2022-05-28,2022-06-04,2022-06-11,2022-06-18,2022-06-25,2022-07-02,2022-07-09,2022-07-16,2022-07-23,2022-07-30,2022-08-06,2022-08-13,2022-08-20,2022-08-27,2022-09-03,2022-09-10,2022-09-17,2022-09-24,2022-10-01,2022-10-08,2022-10-15,2022-10-22,2022-10-29,2022-11-05,2022-11-12,2022-11-19,2022-11-26,2022-12-03,2022-12-10,2022-12-17,2022-12-24,2022-12-31,2023-01-07,2023-01-14,2023-01-21,2023-01-28,2023-02-04,2023-02-11,2023-02-18,2023-02-25,2023-03-04,2023-03-11,2023-03-18,2023-03-25,2023-04-01,2023-04-08,2023-04-15,2023-04-22,2023-04-29,2023-05-06,2023-05-13,2023-05-20,2023-05-27,2023-06-03,2023-06-10,2023-06-17,2023-06-24,2023-07-01,2023-07-08,2023-07-15,2023-07-22,2023-07-29,2023-08-05,2023-08-12,2023-08-19,2023-08-26,2023-09-02,2023-09-09,2023-09-16,2023-09-23,2023-09-30,2023-10-07,2023-10-14,2023-10-21,2023-10-28,2023-11-04,2023-11-11,2023-11-18,2023-11-25,2023-12-02,2023-12-09,2023-12-16,2023-12-23,2023-12-30,2024-01-06,2024-01-13,2024-01-20,2024-01-27,2024-02-03,2024-02-10,2024-02-17,2024-02-24,2024-03-02,2024-03-09,2024-03-16,2024-03-23,2024-03-30,2024-04-06
0,102001,0,United States,country,NaN,78264.0,84017.0,88626.0,91070.0,96028.0,100342.0,104658.0,109072.0,105969.0,108950.0,112388.0,116318.0,121276.0,122980.0,122756.0,123172.0,124740.0,117541.0,119818.0,118179.0,117462.0,121901.0,116465.0,116678.0,114676.0,111545.0,111150.0,107380.0,104230.0,101674.0,98531.0,96062.0,94737.0,93794.0,92478.0,92412.0,90429.0,88755.0,87101.0,84101.0,82187.0,78853.0,69777.0,65423.0,61431.0,57441.0,57465.0,48141.0,45532.0,49355.0,NaN,71977.0,77816.0,81356.0,82525.0,83932.0,89797.0,93337.0,96057.0,101500.0,103274.0,109871.0,114622.0,115604.0,118999.0,119448.0,118623.0,122678.0,121381.0,116510.0,118039.0,114185.0,116564.0,117131.0,108198.0,110179.0,106337.0,103981.0

In [39]:
new_listings_nc = new_listings_data[new_listings_data['StateName']=='NC']
new_listings_nc

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-02-03,2018-02-10,2018-02-17,2018-02-24,2018-03-03,2018-03-10,2018-03-17,2018-03-24,2018-03-31,2018-04-07,2018-04-14,2018-04-21,2018-04-28,2018-05-05,2018-05-12,2018-05-19,2018-05-26,2018-06-02,2018-06-09,2018-06-16,2018-06-23,2018-06-30,2018-07-07,2018-07-14,2018-07-21,2018-07-28,2018-08-04,2018-08-11,2018-08-18,2018-08-25,2018-09-01,2018-09-08,2018-09-15,2018-09-22,2018-09-29,2018-10-06,2018-10-13,2018-10-20,2018-10-27,2018-11-03,2018-11-10,2018-11-17,2018-11-24,2018-12-01,2018-12-08,2018-12-15,2018-12-22,2018-12-29,2019-01-05,2019-01-12,2019-01-19,2019-01-26,2019-02-02,2019-02-09,2019-02-16,2019-02-23,2019-03-02,2019-03-09,2019-03-16,2019-03-23,2019-03-30,2019-04-06,2019-04-13,2019-04-20,2019-04-27,2019-05-04,2019-05-11,2019-05-18,2019-05-25,2019-06-01,2019-06-08,2019-06-15,2019-06-22,2019-06-29,2019-07-06,2019-07-13,2019-07-20,2019-07-27,2019-08-03,2019-08-10,2019-08-17,2019-08-24,2019-08-31,2019-09-07,2019-09-14,2019-09-21,2019-09-28,2019-10-05,2019-10-12,2019-10-19,2019-10-26,2019-11-02,2019-11-09,2019-11-16,2019-11-23,2019-11-30,2019-12-07,2019-12-14,2019-12-21,2019-12-28,2020-01-04,2020-01-11,2020-01-18,2020-01-25,2020-02-01,2020-02-08,2020-02-15,2020-02-22,2020-02-29,2020-03-07,2020-03-14,2020-03-21,2020-03-28,2020-04-04,2020-04-11,2020-04-18,2020-04-25,2020-05-02,2020-05-09,2020-05-16,2020-05-23,2020-05-30,2020-06-06,2020-06-13,2020-06-20,2020-06-27,2020-07-04,2020-07-11,2020-07-18,2020-07-25,2020-08-01,2020-08-08,2020-08-15,2020-08-22,2020-08-29,2020-09-05,2020-09-12,2020-09-19,2020-09-26,2020-10-03,2020-10-10,2020-10-17,2020-10-24,2020-10-31,2020-11-07,...,2021-05-29,2021-06-05,2021-06-12,2021-06-19,2021-06-26,2021-07-03,2021-07-10,2021-07-17,2021-07-24,2021-07-31,2021-08-07,2021-08-14,2021-08-21,2021-08-28,2021-09-04,2021-09-11,2021-09-18,2021-09-25,2021-10-02,2021-10-09,2021-10-16,2021-10-23,2021-10-30,2021-11-06,2021-11-13,2021-11-20,2021-11-27,2021-12-04,2021-12-11,2021-12-18,2021-12-25,2022-01-01,2022-01-08,2022-01-15,2022-01-22,2022-01-29,2022-02-05,2022-02-12,2022-02-19,2022-02-26,2022-03-05,2022-03-12,2022-03-19,2022-03-26,2022-04-02,2022-04-09,2022-04-16,2022-04-23,2022-04-30,2022-05-07,2022-05-14,2022-05-21,2022-05-28,2022-06-04,2022-06-11,2022-06-18,2022-06-25,2022-07-02,2022-07-09,2022-07-16,2022-07-23,2022-07-30,2022-08-06,2022-08-13,2022-08-20,2022-08-27,2022-09-03,2022-09-10,2022-09-17,2022-09-24,2022-10-01,2022-10-08,2022-10-15,2022-10-22,2022-10-29,2022-11-05,2022-11-12,2022-11-19,2022-11-26,2022-12-03,2022-12-10,2022-12-17,2022-12-24,2022-12-31,2023-01-07,2023-01-14,2023-01-21,2023-01-28,2023-02-04,2023-02-11,2023-02-18,2023-02-25,2023-03-04,2023-03-11,2023-03-18,2023-03-25,2023-04-01,2023-04-08,2023-04-15,2023-04-22,2023-04-29,2023-05-06,2023-05-13,2023-05-20,2023-05-27,2023-06-03,2023-06-10,2023-06-17,2023-06-24,2023-07-01,2023-07-08,2023-07-15,2023-07-22,2023-07-29,2023-08-05,2023-08-12,2023-08-19,2023-08-26,2023-09-02,2023-09-09,2023-09-16,2023-09-23,2023-09-30,2023-10-07,2023-10-14,2023-10-21,2023-10-28,2023-11-04,2023-11-11,2023-11-18,2023-11-25,2023-12-02,2023-12-09,2023-12-16,2023-12-23,2023-12-30,2024-01-06,2024-01-13,2024-01-20,2024-01-27,2024-02-03,2024-02-10,2024-02-17,2024-02-24,2024-03-02,2024-03-09,2024-03-16,2024-03-23,2024-03-30,2024-04-06
23,394458,23,"Charlotte, NC",msa,NC,644.0,712.0,783.0,808.0,868.0,901.0,927.0,1037.0,1011.0,1007.0,1051.0,1028.0,1075.0,1118.0,1113.0,1102.0,1106.0,1041.0,1037.0,1030.0,1037.0,1071.0,1029.0,1025.0,1018.0,1006.0,1016.0,994.0,952.0,929.0,895.0,857.0,766.0,728.0,700.0,683.0,716.0,714.0,697.0,685.0,688.0,659.0,596.0,550.0,510.0,459.0,444.0,372.0,351.0,409.0,504.0,617.0,685.0,716.0,734.0,745.0,797.0,835.0,860.0,943.0,955.0,1007.0,1034.0,1001.0,993.0,987.0,996.0,1032.0,1111.0,1056.0,1054.0,1010.0,970.0,983.0,911.0,951.0,925.0,920.0,952.0,926.0,898.0,884.0,861.0,814.0,797.0,801.0,812.0,832.0,827.0,798.0,778.0,719.0,686.0,655.0,609.0,550.0,514.0,468.0,426.0,386.0,350.0,409.0,461.0,585.0,665.0,681.0,

In [40]:
url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"zpid":"2080998890"}

headers = {
	"X-RapidAPI-Key": "9e7be70253msh1047678f06f3800p1319c7jsn7c69aec160d9",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'You are not subscribed to this API.'}
